# AI Config Segments - Cookbook

This cookbook contains all code from `aiconfig-segments/SKILL.md` for managing user segments.

## Prerequisites
- `LAUNCHDARKLY_API_TOKEN`: API token with `segments:write` permission
- Project created (run `cookbook_aiconfig_projects.ipynb` first)

In [1]:
# Install and load environment
%pip install requests python-dotenv -q

import os
from pathlib import Path
from dotenv import load_dotenv

def find_repo_root(start_path: Path = None) -> Path:
    current = start_path or Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / '.git').exists():
            return parent
    return current

repo_root = find_repo_root()
load_dotenv(repo_root / '.env')
print(f"[OK] Loaded environment from {repo_root / '.env'}")


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[OK] Loaded environment from /Users/ld_scarlett/Documents/Github/agent-skills/.env


---
## Complete Segments Management Class
From: `SKILL.md` lines 32-217

In [ ]:
import requests
import os
import time
from typing import Dict, List, Optional, Any

class AIConfigSegments:
    """Manage segments for AI Config targeting"""

    def __init__(self, api_token: str, project_key: str, environment: str = "production"):
        self.api_token = api_token
        self.project_key = project_key
        self.environment = environment
        self.base_url = "https://app.launchdarkly.com/api/v2"
        self.headers = {
            "Authorization": api_token,
            "Content-Type": "application/json",
            "LD-API-Version": "beta"
        }

    def create_segment(self, key: str, name: str, description: str = None, tags: List[str] = None) -> Dict:
        """
        Create a new segment for targeting.

        Args:
            key: Unique segment identifier (lowercase, hyphens)
            name: Human-readable segment name
            description: Optional description
            tags: Optional tags for organization

        Returns:
            Created segment details
        """
        url = f"{self.base_url}/segments/{self.project_key}/{self.environment}"

        payload = {
            "key": key,
            "name": name
        }

        if description:
            payload["description"] = description
        if tags:
            payload["tags"] = tags

        response = requests.post(url, headers=self.headers, json=payload)

        if response.status_code in [200, 201]:
            print(f"[OK] Created segment '{key}'")
            time.sleep(0.5)
            return response.json()
        elif response.status_code == 409:
            print(f"[WARNING] Segment '{key}' already exists")
            return self.get_segment(key)
        else:
            print(f"[ERROR] Failed to create segment: {response.text}")
            return None

    def add_segment_rules(self, segment_key: str, rules: List[Dict]) -> Dict:
        """
        Add targeting rules to a segment.

        Args:
            segment_key: The segment to update
            rules: List of rule clauses

        Returns:
            Updated segment details
        """
        url = f"{self.base_url}/segments/{self.project_key}/{self.environment}/{segment_key}"

        # Build semantic patch instructions
        instructions = []

        for rule in rules:
            clauses = []
            for clause in rule.get("clauses", []):
                clauses.append({
                    "attribute": clause["attribute"],
                    "op": clause["op"],
                    "values": clause["values"],
                    "contextKind": clause.get("contextKind", "user"),
                    "negate": clause.get("negate", False)
                })

            if clauses:
                instructions.append({
                    "kind": "addRule",
                    "clauses": clauses
                })

        payload = {
            "environmentKey": self.environment,
            "instructions": instructions
        }

        # Use semantic patch headers
        patch_headers = self.headers.copy()
        patch_headers["Content-Type"] = "application/json; domain-model=launchdarkly.semanticpatch"

        response = requests.patch(url, headers=patch_headers, json=payload)

        if response.status_code == 200:
            print(f"[OK] Added {len(instructions)} rules to segment '{segment_key}'")
            return response.json()
        else:
            print(f"[ERROR] Failed to add rules: {response.text}")
            return None

    def get_segment(self, segment_key: str) -> Dict:
        """Get segment details"""
        url = f"{self.base_url}/segments/{self.project_key}/{self.environment}/{segment_key}"

        response = requests.get(url, headers=self.headers)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"[ERROR] Failed to get segment: {response.text}")
            return None

    def list_segments(self, limit: int = 50) -> List[Dict]:
        """List all segments in the environment"""
        url = f"{self.base_url}/segments/{self.project_key}/{self.environment}"

        params = {"limit": limit}
        response = requests.get(url, headers=self.headers, params=params)

        if response.status_code == 200:
            data = response.json()
            segments = data.get("items", [])

            print(f"[INFO] Found {len(segments)} segments:")
            for segment in segments:
                rule_count = len(segment.get("rules", []))
                print(f"   - {segment['name']} ({segment['key']}) - {rule_count} rules")

            return segments
        else:
            print(f"[ERROR] Failed to list segments: {response.text}")
            return []

    def delete_segment(self, segment_key: str) -> bool:
        """Delete a segment"""
        url = f"{self.base_url}/segments/{self.project_key}/{self.environment}/{segment_key}"

        response = requests.delete(url, headers=self.headers, json={})

        if response.status_code == 204:
            print(f"[OK] Deleted segment '{segment_key}'")
            return True
        else:
            print(f"[ERROR] Failed to delete segment: {response.text}")
            return False

# Initialize - use same project as aiconfig-create cookbook
API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")
PROJECT_KEY = "support-ai"

segments = AIConfigSegments(API_TOKEN, PROJECT_KEY, "production")
print("[OK] AIConfigSegments class initialized")

In [3]:
# Test: List existing segments and create a test segment
print("=== Testing AIConfigSegments class ===")

# List existing segments
existing_segments = segments.list_segments()

# Create a test segment for the cookbook
TEST_SEGMENT_KEY = "cookbook-beta-testers"
test_segment = segments.create_segment(
    key=TEST_SEGMENT_KEY,
    name="Cookbook Beta Testers",
    description="Test segment created by cookbook",
    tags=["cookbook", "test"]
)

if test_segment:
    print(f"[OK] Test segment ready: {TEST_SEGMENT_KEY}")
else:
    print(f"[INFO] Using existing segment: {TEST_SEGMENT_KEY}")

=== Testing AIConfigSegments class ===
[INFO] Found 1 segments:
   - t (t) - 0 rules
[OK] Created segment 'cookbook-beta-testers'
[OK] Test segment ready: cookbook-beta-testers


---
## Test: Add Rules to Segment
Using the class methods to add rules to our test segment.

In [4]:
# Test: Add targeting rules to our test segment
print("=== Testing add_segment_rules() ===")

result = segments.add_segment_rules(
    TEST_SEGMENT_KEY,
    [
        {
            "clauses": [
                {
                    "attribute": "tier",
                    "op": "in",
                    "values": ["premium", "enterprise"],
                    "contextKind": "user"
                }
            ]
        }
    ]
)

if result:
    print(f"[OK] add_segment_rules() works")
else:
    print("[INFO] Rules may already exist or segment not found")

=== Testing add_segment_rules() ===
[OK] Added 1 rules to segment 'cookbook-beta-testers'
[OK] add_segment_rules() works


---
## Add Individual Contexts to Segment
From: `SKILL.md` lines 591-633

In [ ]:
def add_contexts_to_segment(segment_key: str, context_keys: List[str]):
    """Add specific contexts to a segment"""

    API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")
    PROJECT_KEY = "support-ai"
    ENVIRONMENT = "production"

    url = f"https://app.launchdarkly.com/api/v2/segments/{PROJECT_KEY}/{ENVIRONMENT}/{segment_key}"

    instructions = [
        {
            "kind": "addIncludedTargets",
            "contextKind": "user",
            "values": context_keys
        }
    ]

    payload = {
        "environmentKey": ENVIRONMENT,
        "instructions": instructions
    }

    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json; domain-model=launchdarkly.semanticpatch"
    }

    response = requests.patch(url, headers=headers, json=payload)

    if response.status_code == 200:
        print(f"[OK] Added {len(context_keys)} contexts to segment")
        return response.json()
    else:
        print(f"[ERROR] Failed to add contexts: {response.text}")
        return None

# Test with our cookbook segment
print("=== Testing add_contexts_to_segment() ===")
result = add_contexts_to_segment(
    TEST_SEGMENT_KEY,
    ["cookbook-user-1", "cookbook-user-2"]
)
if result:
    print(f"[OK] add_contexts_to_segment() works")

---
## Exclude Contexts from Segment
From: `SKILL.md` lines 637-679

In [ ]:
def exclude_contexts_from_segment(segment_key: str, context_keys: List[str]):
    """Exclude specific contexts from a segment"""

    API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")
    PROJECT_KEY = "support-ai"
    ENVIRONMENT = "production"

    url = f"https://app.launchdarkly.com/api/v2/segments/{PROJECT_KEY}/{ENVIRONMENT}/{segment_key}"

    instructions = [
        {
            "kind": "addExcludedTargets",
            "contextKind": "user",
            "values": context_keys
        }
    ]

    payload = {
        "environmentKey": ENVIRONMENT,
        "instructions": instructions
    }

    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json; domain-model=launchdarkly.semanticpatch"
    }

    response = requests.patch(url, headers=headers, json=payload)

    if response.status_code == 200:
        print(f"[OK] Excluded {len(context_keys)} contexts from segment")
        return response.json()
    else:
        print(f"[ERROR] Failed to exclude contexts: {response.text}")
        return None

# Test with our cookbook segment
print("=== Testing exclude_contexts_from_segment() ===")
result = exclude_contexts_from_segment(
    TEST_SEGMENT_KEY,
    ["excluded-user-1"]
)
if result:
    print(f"[OK] exclude_contexts_from_segment() works")